In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('/home/carlo2/workspace/networkqit/')
import networkqit as nq
import numpy as np
import matplotlib.pyplot as plt
import bct

In [ ]:
W=np.loadtxt('/home/carlo2/workspace/communityalg/data/GroupAverage_rsfMRI_unthr.adj')
t=0.6
Wt = bct.threshold_absolute(W,t)[0:32,0:32]

In [ ]:
W=np.random.geometric(0.2,size=(32,32))
W=np.triu(W,1)
W=W+W.T
Wt=bct.threshold_absolute(W,6)
Wt = np.triu(Wt,1)
Wt += Wt.T
plt.imshow(Wt)
plt.colorbar()

In [ ]:
M=nq.CWTECM2(N=len(Wt),threshold=Wt[np.nonzero(Wt)].min())
sol = M.fit(G=Wt, method='MLE')#, ftol=1E-15, gtol=1E-12,verbose=0,maxiter=5000)

In [ ]:
nq.plot_mle(Wt,M.expected_adjacency(sol['x']), M.expected_weighted_adjacency(sol['x']))
plt.show()

In [ ]:
nq.plot_mle(Wt,M.expected_adjacency(sol['x']), np.mean(M.sample_adjacency(sol['x'],batch_size=100),0))
plt.show()

In [ ]:
X=M.sample_adjacency(sol['x'],batch_size=100)
_=plt.hist(X[np.nonzero(X)],200)
print(np.min(X[np.nonzero(X)]))